# Project 1: Digit Classification with KNN and Naive Bayes

In this project, you'll implement your own image recognition system for classifying digits. Read through the code and the instructions carefully and add your own code where indicated. Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but <b> please prepare your own write-up (with your own code). </b>

If you're interested, check out these links related to digit recognition:

* Yann Lecun's MNIST benchmarks: http://yann.lecun.com/exdb/mnist/
* Stanford Streetview research and data: http://ufldl.stanford.edu/housenumbers/

Finally, if you'd like to get started with Tensorflow, you can read through this tutorial: https://www.tensorflow.org/tutorials/keras/basic_classification. It uses a dataset called "fashion_mnist", which is identical in structure to the original digit mnist, but uses images of clothing rather than images of digits. The number of training examples and number of labels is the same. In fact, you can simply replace the code that loads "fashion_mnist" with simply "mnist" and everything should work fine.

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

Load the data. Notice that we are splitting the data into training, development, and test. We also have a small subset of the training data called mini_train_data and mini_train_labels that you should use in all the experiments below, unless otherwise noted.

In [ ]:
# Load the digit data from https://www.openml.org/d/554 or from default local location '~/scikit_learn_data/...'
X, Y = fetch_openml(name='mnist_784', return_X_y=True, cache=False)


# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]
print(mini_train_data.shape)
print(mini_train_data[1].shape)

(1) Create a 10x10 grid to visualize 10 examples of each digit. Python hints:

- plt.rc() for setting the colormap, for example to black and white
- plt.subplot() for creating subplots
- plt.imshow() for rendering a matrix
- np.array.reshape() for reshaping a 1D feature vector into a 2D matrix (for rendering)

In [ ]:
train_labels[0] == str(0)

In [ ]:
def P1(num_examples=10):
  ### STUDENT START ###
  figure, axes = plt.subplots(10, 10)
  for x in range(0, 10):
    for y in range(0, 10):
      for z in range(0, len(train_labels)):
        num = z
        if train_labels[num] == str(x):
          axes[x,y].imshow(train_data[z].reshape(28, 28),cmap='gray_r')
          plt.subplots_adjust(bottom=0.2, right=1.5, top=1.5)
          axes[x,y].axes.set_frame_on(True)
          axes[x,y].set_xticks([], [])
          axes[x,y].set_yticks([], [])
          break
  ### STUDENT END ###

P1(10)

(2) Evaluate a K-Nearest-Neighbors model with k = [1,3,5,7,9] using the mini training set. Report accuracy on the dev set. For k=1, show precision, recall, and F1 for each label. Which is the most difficult digit?

- KNeighborsClassifier() for fitting and predicting
- classification_report() for producing precision, recall, F1 results

In [ ]:
def P2(k_values):

### STUDENT START ###
  for k in k_values:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(mini_train_data, mini_train_labels)
    train_predicted_labels = model.predict(dev_data)
    print('Results for model k = %d'%k)
    print(classification_report(dev_labels,train_predicted_labels))

    
### STUDENT END ###

k_values = [1, 3, 5, 7, 9]
P2(k_values)


ANSWER: The most difficult digit to predict is 8.

(3) Using k=1, report dev set accuracy for the training set sizes below. Also, measure the amount of time needed for prediction with each training size.

- time.time() gives a wall clock value you can use for timing operations

In [ ]:
len(dev_data)

In [ ]:
def P3(train_sizes, accuracies):

### STUDENT START ###
  total_time = []
  loop_count = 0
  for t in train_sizes:
    model = KNeighborsClassifier(n_neighbors=1)
    model.fit(train_data[:t], train_labels[:t])
    start_time = time.time()
    train_predicted_labels = model.predict(dev_data)
    end_time = time.time()
    total_time.append(end_time - start_time)
    accuracies.append(sum(dev_labels == train_predicted_labels)/len(dev_data))
    print('The accuracy for ' + str(t)+ ' is '+str(accuracies[loop_count]*100)+' percent and took a total time of '+str(total_time[loop_count])+' seconds.')
    loop_count += 1
  return(accuracies,total_time)

### STUDENT END ###

train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25000]
accuracies = []
P3(train_sizes, accuracies)

(4) Fit a regression model that predicts accuracy from training size. What does it predict for n=60000? What's wrong with using regression here? Can you apply a transformation that makes the predictions more reasonable?

- Remember that the sklearn fit() functions take an input matrix X and output vector Y. So each input example in X is a vector, even if it contains only a single value.

In [ ]:
def P4(sample_data,train_sizes,n):

### STUDENT START ###
  regr = LinearRegression()
  acc_x = np.array(train_sizes)
  acc_x = np.reshape(acc_x,(-1,1),)
  acc_y = np.array(sample_data)
  acc_y = np.reshape(acc_y,(-1,1))
  regr.fit(acc_x,acc_y)
  arr_n = np.array(n)
  arr_n = np.reshape(arr_n,(1,-1))
  prediction = regr.predict(arr_n)
  return(prediction)

### STUDENT END ###
n = 60000
standard_P4 = P4(accuracies,train_sizes,n)
log_P4 = P4(np.log(accuracies),np.log(train_sizes),np.log(n))
print(standard_P4)
print(np.exp(log_P4))

ANSWER: There are a couple of problems here, first 60000 was above our training data so the prediction would be to the right of the "known universe" this is often a cause of error.  The other and more glaring problem is that the prediction is for 124% accuracy. Obviously this is not possible.  The data could be transformed using logs which should make it a better predictor.

Fit a 1-NN and output a confusion matrix for the dev data. Use the confusion matrix to identify the most confused pair of digits, and display a few example mistakes.

- confusion_matrix() produces a confusion matrix

In [ ]:
def P5(a):
### STUDENT START ###
  model = KNeighborsClassifier(n_neighbors=a)
  model.fit(mini_train_data, mini_train_labels)
  train_predicted_labels = model.predict(dev_data)
  conf = confusion_matrix(dev_labels,train_predicted_labels)
  print(conf)
  plt.imshow(conf, cmap='binary', interpolation='None')
  plt.show()
  l = 0
  b = 0
  while (l < 5):
    if (train_predicted_labels[b] != dev_labels[b] and dev_labels[b]=='4' and train_predicted_labels[b] == '9'):
      print('This was predicted to be a:' + str(train_predicted_labels[b]))
      plt.imshow(dev_data[b].reshape(28, 28),cmap='gray_r')
      plt.show()
      l += 1
    b += 1
    if b == 1000:
      break


  ### STUDENT END ### 


P5(1)


ANSWER: The most common error is confusing 4 for a 9.  We also see errors on 9s confused with 7s.

(6) A common image processing technique is to smooth an image by blurring. The idea is that the value of a particular pixel is estimated as the weighted combination of the original value and the values around it. Typically, the blurring is Gaussian -- that is, the weight of a pixel's influence is determined by a Gaussian function over the distance to the relevant pixel.

Implement a simplified Gaussian blur by just using the 8 neighboring pixels: the smoothed value of a pixel is a weighted combination of the original value and the 8 neighboring values. Try applying your blur filter in 3 ways:
- preprocess the training data but not the dev data
- preprocess the dev data but not the training data
- preprocess both training and dev data

Note that there are Guassian blur filters available, for example in scipy.ndimage.filters. You're welcome to experiment with those, but you are likely to get the best results with the simplified version I described above.

In [ ]:
def P6(data_set):
    
### STUDENT START ###
  result = np.zeros(data_set.shape)
  w = 0.1
  for b in range(data_set.shape[0]):
    grid = data_set[b].reshape(28, 28)
    f = np.zeros((28, 28))
    for i in range(28):
      for j in range(28):
        f[i,j] = grid[i,j] + np.sum(grid[i-1:i+2, j-1:j+2]) / w
    result[b] = f.reshape(784,)
  return(result)

### STUDENT END ###

def knn(train_y,test_x):
  clf = KNeighborsClassifier()
  clf.fit(train_y,mini_train_labels)
  predicts = clf.predict(test_x)
  print('The accuracy of this method is '+str(sum(predicts == dev_labels)/len(dev_labels)))

blur_train_data = P6(mini_train_data)
blur_dev_data = P6(dev_data)

print('Standard unblurred data')
knn(mini_train_data,dev_data)

print('Blurring the training data but not the dev data')
knn(blur_train_data,dev_data)

print('Blurring both the training data and the dev data')
knn(blur_train_data,blur_dev_data)

ANSWER: When applying the Gaussian blur to the training and the dev data it significantly lowered the accuracy when blurring training but not dev. However, when the blurring was applied to both sets, the accuracy increased approximately 3 percentage points which crossed the 90% barrier.

(7) Fit a Naive Bayes classifier and report accuracy on the dev data. Remember that Naive Bayes estimates P(feature|label). While sklearn can handle real-valued features, let's start by mapping the pixel values to either 0 or 1. You can do this as a preprocessing step, or with the binarize argument. With binary-valued features, you can use BernoulliNB. Next try mapping the pixel values to 0, 1, or 2, representing white, grey, or black. This mapping requires MultinomialNB. Does the multi-class version improve the results? Why or why not?

In [ ]:
def create_bin(data_set,threshold):
  bin_data_set = np.zeros(data_set.shape)
  for b in range(data_set.shape[0]):
    grid = data_set[b].reshape(28, 28)
    f = np.zeros((28, 28))
    for i in range(28):
      for j in range(28):
        #f[i,j] = 1
        if grid[i,j] < threshold:
          f[i,j] = 0
        else:
          f[i,j] = 1
    bin_data_set[b] = f.reshape(784,)
  return(bin_data_set)

def create_012(data_set,lower_thresh,upper_thresh):
  bin_data_set = np.zeros(data_set.shape)
  for b in range(data_set.shape[0]):
    grid = data_set[b].reshape(28, 28)
    f = np.zeros((28, 28))
    for i in range(28):
      for j in range(28):
        #f[i,j] = 1
        if grid[i,j] < lower_thresh:
          f[i,j] = 0
        elif grid[i,j] > upper_thresh:
          f[i,j] = 2
        else:
          f[i,j] = 1
    bin_data_set[b] = f.reshape(784,)
  return(bin_data_set)

def bernoulli(data_set,threshold):
  clf = BernoulliNB()
  bin_train_data = create_bin(mini_train_data,threshold)
  clf.fit(bin_train_data,mini_train_labels)
  return(clf.predict(data_set))

def multinomial(data_set,lower_thresh,upper_thresh):
  clf = MultinomialNB()
  mult_train_data = create_012(mini_train_data,lower_thresh,upper_thresh)
  clf.fit(mult_train_data,mini_train_labels)
  return(clf.predict(data_set))


def P7_bern(data_set,threshold):
  bin_data_P7 = create_bin(data_set,threshold)
  P7_predict = bernoulli(bin_data_P7,threshold)
  return(P7_predict)

def P7_mult(data_set,lower_thresh,upper_thresh):
  mult_data_P7 = create_012(data_set,lower_thresh,upper_thresh)
  P7_predict = multinomial(mult_data_P7,lower_thresh,upper_thresh)
  return(P7_predict)
    
### STUDENT END ###

bern_predict = P7_bern(dev_data,0.4)
print('The BernoulliNB method created '+str(sum(dev_labels != bern_predict))+' errors.')

mult_predict = P7_mult(dev_data,0.3,0.6)
print('The MultinomialNB method created '+str(sum(dev_labels != mult_predict))+' errors.')


ANSWER: The multinomial classification does not improve errors.  Generally this should be a binary decision on a pixel by pixel basis.  The multinomial likley overfits on the training data.

(8) Use GridSearchCV to perform a search over values of alpha (the Laplace smoothing parameter) in a Bernoulli NB model. What is the best value for alpha? What is the accuracy when alpha=0? Is this what you'd expect?

- Note that GridSearchCV partitions the training data so the results will be a bit different than if you used the dev data for evaluation.

In [ ]:
def create_bin(data_set):
  bin_data_set = np.zeros(data_set.shape)
  for b in range(data_set.shape[0]):
    grid = data_set[b].reshape(28, 28)
    f = np.zeros((28, 28))
    for i in range(28):
      for j in range(28):
        #f[i,j] = 1
        if grid[i,j] < 0.4:
          f[i,j] = 0
        else:
          f[i,j] = 1
    bin_data_set[b] = f.reshape(784,)
  return(bin_data_set)

def P8(alphas):

### STUDENT START ###
  clf = GridSearchCV(BernoulliNB(), param_grid=alphas, refit = True)
  return(clf.fit(bin_train_data,mini_train_labels))



### STUDENT END ###
bin_train_data = create_bin(mini_train_data)
bin_dev_data = create_bin(dev_data)
#pipeline = Pipeline([('classifier', BernoulliNB())])
alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
nb = P8(alphas)
print(nb.cv_results_['mean_test_score'])

In [ ]:
print(nb.best_params_)
clf_0 = GridSearchCV(BernoulliNB(),param_grid={'alpha':[0.0]}, refit = True)
clf_0.fit(bin_train_data,mini_train_labels)
pred_clf_0 = clf_0.best_estimator_.predict(bin_dev_data)
print(sum(pred_clf_0 != dev_labels))

*ANSWER*: The best answer for alpha was 0.0001. When running the same process over alpha = 0.0 the classification made 183 errors. This is expected as having alpha set to zero should behave the same as running the BernoulliNB process in previous questions.

NOTE: I tried pipelining the data for ages, but could not get the system to process without an error.  This clearly is less efficient.

(9) Try training a model using GuassianNB, which is intended for real-valued features, and evaluate on the dev data. You'll notice that it doesn't work so well. Try to diagnose the problem. You should be able to find a simple fix that returns the accuracy to around the same rate as BernoulliNB. Explain your solution.

Hint: examine the parameters estimated by the fit() method, theta\_ and sigma\_.

In [ ]:
def P9(test_vars):

### STUDENT END ###
  clf = GaussianNB(priors=None, var_smoothing=test_vars)
  clf_9 = clf.fit(mini_train_data,mini_train_labels)
  print(clf_9.get_params)
  return(clf_9.predict(dev_data))


### STUDENT END ###
#some_params = 'sigma_0.2'


test_vars = [0.01,0.02,0.05,0.075,0.1,0.5]
#gnb = P9()
#print(sum(dev_labels != gnb))
for i in test_vars:
  tt = P9(i)
  print(sum(tt != dev_labels))

*ANSWER*: In my first attempt, I fit the model on the mini_train_data and predicted on the dev_data.  I did not binarize the data because of the statement that GaussianNB is intended for real value features.  However, the error rate in an uncorrected model was 407 / 1000.

To get the mdoel working more closely to the BernoulliNB model, I updated the var_smoothing variable.  This basically allows for a greater variance tolerance than the default.

(10) Because Naive Bayes is a generative model, we can use the trained model to generate digits. Train a BernoulliNB model and then generate a 10x20 grid with 20 examples of each digit. Because you're using a Bernoulli model, each pixel output will be either 0 or 1. How do the generated digits compare to the training digits?

- You can use np.random.rand() to generate random numbers from a uniform distribution
- The estimated probability of each pixel is stored in feature\_log\_prob\_. You'll need to use np.exp() to convert a log probability back to a probability.

In [ ]:
def P10(num_examples):

### STUDENT START ###
  nb = BernoulliNB()
  nb.fit(mini_train_data,mini_train_labels)

  fig, axes = plt.subplots(10, num_examples, figsize=(20, 10))
  plt.rc('image', cmap='binary')

  for i in range(10):
    for j in range(num_examples):
      log_probs = nb.feature_log_prob_[i]
      n = log_probs.shape[0]
      r = np.random.rand(n)
      d = np.zeros(n)
      for k in range(n):
        if r[k] < np.exp(log_probs[k]):
          d[k] = 1
        else:
          d[k] = 0
      axes[i][j].imshow(d.reshape(28,28), cmap = 'gray_r')
      axes[i][j].axis('off')





### STUDENT END ###

P10(20)

ANSWER: Generally the digits look pretty good, but you can tell which digits have a lower general probability throught, like 4.  4 is quite fuzzy while 0 and 1 are clear.

(11) Remember that a strongly calibrated classifier is rougly 90% accurate when the posterior probability of the predicted class is 0.9. A weakly calibrated classifier is more accurate when the posterior is 90% than when it is 80%. A poorly calibrated classifier has no positive correlation between posterior and accuracy.

Train a BernoulliNB model with a reasonable alpha value. For each posterior bucket (think of a bin in a histogram), you want to estimate the classifier's accuracy. So for each prediction, find the bucket the maximum posterior belongs to and update the "correct" and "total" counters.

How would you characterize the calibration for the Naive Bayes model?

In [ ]:
def P11(buckets, correct, total):
    
### STUDENT START ###
  nb = BernoulliNB(alpha=0.01)
  nb.fit(bin_train_data,mini_train_labels)
  predict_nb = nb.predict_proba(bin_dev_data)

  for b in range(len(buckets)):
    for c in range(0,predict_nb.shape[0]):
      if (np.max(predict_nb[c]) >= buckets[b]):
        total[b] += 1
        if (np.argmax(predict_nb[c]) == np.argmax(dev_labels[c])):
          correct[b] += 1




                
### STUDENT END ###

buckets = [0.5, 0.9, 0.999, 0.99999, 0.9999999, 0.999999999, 0.99999999999, 0.9999999999999, 1.0]
correct = [0 for i in buckets]
total = [0 for i in buckets]

P11(buckets, correct, total)
print(correct)
for i in range(len(buckets)):
    accuracy = 0.0
    if (total[i] > 0): accuracy = correct[i] / total[i]
    print('p(pred) <= %.13f    total = %3d    accuracy = %.3f' %(buckets[i], total[i], accuracy))

ANSWER: This is a pretty poorly calibrated model.  I feel like there's something I'm still getting wrong here. I've tried numerous variations of the alpha and using binarize versus the binary function written earlier.

(12) EXTRA CREDIT

Try designing extra features to see if you can improve the performance of Naive Bayes on the dev set. Here are a few ideas to get you started:
- Try summing the pixel values in each row and each column.
- Try counting the number of enclosed regions; 8 usually has 2 enclosed regions, 9 usually has 1, and 7 usually has 0.

Make sure you comment your code well!

In [ ]:
#def P12():

### STUDENT START ###


### STUDENT END ###

#P12()